<a href="https://colab.research.google.com/github/Rachel-0420/Cross-Entropy-Fast-Transient/blob/real-data/cross_entropy_btw_supernovae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install astropy pzflow corner

from pzflow import Flow
import jax.numpy as jnp
import pandas as pd
import numpy as np
import pickle
import corner
import matplotlib.pyplot as plt
import os
from astropy.table import Table
from pzflow import Flow, FlowEnsemble
from pzflow.distributions import Uniform
from pzflow.bijectors import Chain, StandardScaler, NeuralSplineCoupling, ColorTransform, InvSoftplus, RollingSplineCoupling, ShiftBounds
from collections import namedtuple

import scipy.stats as sps

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.0/368.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.2/577.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Paths for the Probability Cubes of some transient classes
# SNII and SNIbc are both supernovae and are similar. AGN is very different from the other two

data1 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_1815__SNII-Templates.pkl"
data2 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_0821__SNIbc-Templates.pkl"
data3 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_1819__AGN.pkl"

In [12]:
with open(data1, 'rb') as f:
    CoordinateDict = pickle.load(f)
    Data = pickle.load(f)

In [13]:
f

<_io.BufferedReader name='/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_1815__SNII-Templates.pkl'>

In [9]:
CoordinateDict

{'EventName': 'SNII-Templates',
 'PointsPerDay': 1,
 'ObjectNo': 1000,
 'BandPairs': ['ug',
  'ur',
  'ui',
  'uz',
  'gu',
  'gr',
  'gi',
  'gz',
  'gy',
  'ru',
  'rg',
  'ri',
  'rz',
  'ry',
  'iu',
  'ig',
  'ir',
  'iz',
  'iy',
  'zu',
  'zg',
  'zr',
  'zi',
  'zy',
  'yg',
  'yr',
  'yi',
  'yz'],
 'dT1s': array([-480, -465, -450, -435, -420, -405, -390, -375, -360, -345, -330,
        -315, -300, -285, -270, -255, -240, -225, -210, -195, -180, -165,
        -150, -135, -120, -105,  -90,  -75,  -60,  -45,  -30,  -15,    0,
          15,   30,   45,   60,   75,   90,  105,  120,  135,  150,  165,
         180,  195,  210,  225,  240,  255,  270,  285,  300,  315,  330,
         345,  360,  375,  390,  405,  420,  435,  450,  465,  480]),
 'dT2s': array([-1920, -1890, -1860, -1830, -1800, -1770, -1740, -1710, -1680,
        -1650, -1620, -1590, -1560, -1530, -1500, -1470, -1440,  -480,
         -450,  -420,  -390,  -360,  -330,  -300,  -270,  -240,  -210,
         -180,  -150, 

In [20]:
Data

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
    